### Compile or Load Graph
Here we'll be interacting with a server that's exposing a chat bot with memmory history being persisted on the backend.

In [1]:
## From LangServe RemoteRunnable
# NOTE: deprecated by command update!
from langserve import RemoteRunnable
from langgraph.types import Command

cheese = RemoteRunnable("http://localhost:8000/cheese")

In [1]:
## From LangGraph
from langgraph.types import Command
from cheese.workflow_builder import WorkflowBuilder
from cheese.config.config_graph import ConfigGraph

# Workflow Configuration
workflow_builder = WorkflowBuilder(config=ConfigGraph)
cheese = workflow_builder.compile() # compile the graph
workflow_builder.display_graph(save=True, filepath="artifacts/cheese_graph.png") # update the graph artifact

### Quick Graph Test

In [2]:
# Define the thread_id
config = {"configurable": {"thread_id": "001"}}

In [ ]:
# Define the input and invoke
user_input = "Ey, ¿qué tal?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
for event in cheese.stream(message_input, config, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
# Define the input and invoke
user_input = "¿Puedes resumirme los últimos 25 mensajes?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
for event in cheese.stream(message_input, config, stream_mode="updates"):
    print(event)
    print("\n")

In [ ]:
for event in cheese.stream(
    # provide feedback
    Command(resume={
        "action": "feedback", 
        "data": "Perdona, quise decir 5 mensajes"
        }
    ),
    config,
    stream_mode="updates",
):
    print(event)
    print("\n")

In [ ]:
for event in cheese.stream(
    # provide continue
    Command(resume={"action": "continue"}),
    config,
    stream_mode="updates",
):
    print(event)
    print("\n")

### Live Graph Test (DEPRECATED)

In [ ]:
## Live chat via stream
# Define the thread_id
config = {"configurable": {"thread_id": "002"}}

while True:
    user_input = input("Human: ")
    message_input = {"messages": [{"role": "human", "content": user_input}]}
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Cheese-Chatter: Adiós!")
        break
    else:
        print(f"Human: {user_input}") # to see the user input in the live chat
    for event in cheese.stream(message_input, config): 
        for key, value in event.items():
            if key == 'cheeseagent' and len(value["messages"][-1].content) > 0:
                print("Cheese-Chatter:", value["messages"][-1].content)     

In [ ]:
## Follow the nodes via stream
# Define the thread_id
config = {"configurable": {"thread_id": "002"}}

# Define the input and invoke
user_input = "Do you know what is an atom?"
message_input = {"messages": [{"role": "human", "content": user_input}]}
for event in cheese.stream(message_input, config):
    # stream() yields dictionaries with output keyed by node name
    for key, value in event.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

### Graph Request API (DEPRECATED)

In [ ]:
## From LangServe API
# NOTE: deprecated by command update!
import requests

response = requests.post(
    url='http://localhost:8000/cheese/invoke',
    json={
        'input': {
            "messages": [("human", "Hola, cómo estás")],
        },
        'config': {
            'configurable': {
                "thread_id": "004"
            }
        }
    }
)

print(response.json()["output"]["messages"][-1])